Let's take the code from the previous notebook and add memory to that agent.

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(".env"))

# Adding Memory

In [16]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.agents import create_react_agent
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import OpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

search = TavilySearchResults()
search.invoke("what is the weather in Portugal?")
loader = WebBaseLoader("https://python.langchain.com/docs/get_started/introduction")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()
retriever.get_relevant_documents("What is langchain?")[0]
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langchain_search",
    "Search for information about LangChain. For any questions about LangChain, you must use this tool!",
)

In [11]:
tools = [search, retriever_tool]

In [18]:
prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [20]:
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory
)

In [24]:
agent_chain.invoke(input="My name is Lucas and I work as a Machine Learning Engineer")



> Entering new AgentExecutor chain...
Thought: I should use the langchain_search tool to find information about LangChain.
Action: langchain_search
Action Input: LangChain
Observation: Introduction | 🦜️🔗 Langchain

rely on a language model to reason (about how to answer based on provided context, what actions to take, etc.)This framework consists of several parts.LangChain Libraries: The Python and JavaScript libraries. Contains interfaces and integrations for a myriad of components, a basic run time for combining these components into chains and agents, and off-the-shelf implementations of chains and agents.LangChain Templates: A collection of easily deployable reference architectures for a wide variety of tasks.LangServe: A library for deploying LangChain chains as a REST API.LangSmith: A developer platform that lets you debug, test, evaluate, and monitor chains built on any LLM framework and seamlessly integrates with LangChain.Together, these products simplify the entire applicat

{'input': 'My name is Lucas and I work as a Machine Learning Engineer',
 'chat_history': 'Human: How many people live in canada?\nAI: According to the most recent data, the population of Canada is approximately 40 million people.\nHuman: How many people live in canada?\nAI: According to the most recent data, the population of Canada is approximately 40 million people.\nHuman: what is their national anthem called?\nAI: The national anthem of Canada is "O Canada".',
 'output': 'LangChain is a framework for building and deploying language models and cognitive architectures. It consists of several components, including the LangChain Libraries, Templates, LangServe, and LangSmith. The LangChain Expression Language (LCEL) is used to compose chains and make it easy to customize and deploy them. LangChain is designed to simplify the application lifecycle, from development to production and deployment.'}

In [25]:
agent_chain.invoke(input="What is a similar job to mine?")



> Entering new AgentExecutor chain...
Thought: I should use a search engine to find similar job titles.
Action: tavily_search_results_json
Action Input: "Similar job titles to Machine Learning Engineer"
Observation: [{'url': 'https://www.projectpro.io/article/machine-learning-engineer-career-path/537', 'content': 'A machine learning expert\nexplores data science prototypes, and designs Machine Learning systems,\nperforms research and implements relevant machine learning algorithms and tools,\ndevelop machine learning applications according to requirements,\nperforms statistical analysis and fine-tuning,\nextends existing machine learning libraries and frameworks, etc.\n A data science expert\ntakes up a business problem and employs various machine learning algorithms to build a solution,\nconducts market research, and identifies patterns in a given dataset,\nbuild deep learning models using frameworks such as Tensorflow, Keras, etc., and\nperforms data analysis and data cleaning acti

{'input': 'What is a similar job to mine?',
 'chat_history': 'Human: How many people live in canada?\nAI: According to the most recent data, the population of Canada is approximately 40 million people.\nHuman: How many people live in canada?\nAI: According to the most recent data, the population of Canada is approximately 40 million people.\nHuman: what is their national anthem called?\nAI: The national anthem of Canada is "O Canada".\nHuman: My name is Lucas and I work as a Machine Learning Engineer\nAI: LangChain is a framework for building and deploying language models and cognitive architectures. It consists of several components, including the LangChain Libraries, Templates, LangServe, and LangSmith. The LangChain Expression Language (LCEL) is used to compose chains and make it easy to customize and deploy them. LangChain is designed to simplify the application lifecycle, from development to production and deployment.',
 'output': 'Some similar job titles to Machine Learning Engin